In [2]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor 
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.svm import SVR
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

import warnings

warnings.filterwarnings('ignore') 

In [3]:
df=pd.read_csv("tourism_sql.csv")

In [29]:
columns_drop=['Continent','Region','AttractionAddress','VisitMode','Country','CityName','Attraction','AttractionType']
df=df.drop(columns=columns_drop)

In [30]:
columns_drop=['UserId',  'VisitYear', 'VisitMonth']
df=df.drop(columns=columns_drop)

In [24]:
df.columns

Index(['ContinentId', 'RegionId', 'CountryId', 'CityId', 'TransactionId',
       'VisitModeId', 'AttractionId', 'AttractionCityId', 'AttractionTypeId',
       'Rating'],
      dtype='object')

In [5]:
# Start with the raw DataFrame
df = pd.read_csv("tourism_sql.csv")

# Drop missing UserId or Rating (required for user-level stats)
df = df.dropna(subset=["UserId", "Rating"])

# Ensure proper types
df["UserId"] = df["UserId"].astype(str)
df["AttractionId"] = df["AttractionId"].astype(str)

# --- 🧑‍💼 User-specific features ---
user_stats = df.groupby("UserId")["Rating"].agg(
    Avg_user_rating="mean",
    User_visit_count="count"
).reset_index()

# --- 🏛️ Attraction-specific features ---
attraction_stats = df.groupby("AttractionId")["Rating"].agg(
    Attraction_avg_rating="mean",
    Attraction_popularity="count"
).reset_index()

# --- 🔗 Merge with original dataframe ---
df = df.merge(user_stats, on="UserId", how="left")
df = df.merge(attraction_stats, on="AttractionId", how="left")


In [35]:
df.head()

,UserId,ContinentId,Continent,RegionId,Region,CountryId,Country,CityId,CityName,TransactionId,...,Attraction,AttractionCityId,AttractionAddress,AttractionTypeId,AttractionType,Rating,Avg_user_rating,User_visit_count,Attraction_avg_rating,Attraction_popularity
0,23,1,Africa,4,Southern Africa,22,South Africa,154,Meyerton,69140,...,Tegalalang Rice Terrace,1,"Jalan Raya Ceking, Tegalalang 80517 Indonesia",72,Points of Interest & Landmarks,5,5.000000,1,4.157079,5806
1,34,1,Africa,4,Southern Africa,22,South Africa,167,Paarl,11243,...,Sacred Monkey Forest Sanctuary,1,"Jl. Monkey Forest, Ubud 80571 Indonesia",63,Nature & Wildlife Areas,4,4.000000,1,4.267207,13192
2,60,1,Africa,3,North Africa,14,Egypt,52,Cairo,67843,...,Tegalalang Rice Terrace,1,"Jalan Raya Ceking, Tegalalang 80517 Indonesia",72,Points of Interest & Landmarks,5,5.000000,1,4.157079,5806
3,103,1,Africa,4,Southern Africa,22,South Africa,106,Cape Town,9492,...,Sacred Monkey Forest Sanctuary,1,"Jl. Monkey Forest, Ubud 80571 Indonesia",63,Nature & Wildlife Areas,4,4.666667,3,4.267207,13192
4,103,1,Africa,4,Southern Africa,22,South Africa,106,Cape Town,25475,...,Waterbom Bali,1,"Jl. Kartika Plaza, Kuta 80361 Indonesia",92,Water Parks,5,4.666667,3,4.646592,6426


In [36]:
# --- Select features ---
#features = ['AttractionId','TransactionId','RegionId','AttractionTypeId', 'AttractionCityId','UserId','VisitYear','VisitMonth']
features = [
    'ContinentId',
    'CountryId',
    'CityId',
    'AttractionTypeId',
    'Avg_user_rating',
    'User_visit_count',
    'Attraction_avg_rating',
    'Attraction_popularity'
]

target = 'Rating'

# --- Prepare data ---
df_model = df[features + [target]].dropna()
X = df_model[features]
y = df_model[target]

# --- Train/test split ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [37]:
# --- Initialize models ---
models = {
    "Decision Tree": DecisionTreeRegressor(max_depth=5, random_state=42),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "XGBoost": XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42, verbosity=0),
    "CatBoost": CatBoostRegressor(iterations=100, learning_rate=0.1, depth=5, verbose=0, random_state=42),
    #"LightGBM": LGBMRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
}

# --- Store metrics ---
results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    results.append({
        "Model": name,
        "MSE": mse,
        "MAE": mae,
        "R2 Score": r2
    })

# --- Convert results to DataFrame ---
results_df = pd.DataFrame(results).sort_values(by="R2 Score", ascending=False)
print(results_df)

           Model       MSE       MAE  R2 Score
3       CatBoost  0.234322  0.260494  0.753707
2        XGBoost  0.239160  0.259621  0.748623
0  Decision Tree  0.244834  0.265337  0.742658
1  Random Forest  0.305285  0.277691  0.679119


In [ ]:
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# --- Features and Target ---
features = [
    'ContinentId',
    'CountryId',
    'CityId',
    'AttractionTypeId',
    'Avg_user_rating',
    'User_visit_count',
    'Attraction_avg_rating',
    'Attraction_popularity'
]
target = 'Rating'

# --- Prepare Data ---
df_model = df[features + [target]].dropna()
X = df_model[features]
y = df_model[target]

# --- Train/Test Split ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Initialize and Train Model ---
model = CatBoostRegressor(
    iterations=100,
    learning_rate=0.1,
    depth=5,
    verbose=0,
    random_state=42
)
model.fit(X_train, y_train)

# --- Predictions ---
y_pred = model.predict(X_test)

# --- Evaluation ---
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.2f}")
print(f"R^2 Score: {r2:.2f}")


Mean Squared Error: 0.23
R^2 Score: 0.75


In [40]:
import pickle

# Save the model to a file
with open("catboost_model_regression.pkl", "wb") as f:
    pickle.dump(model, f)


In [41]:

from sklearn.model_selection import RandomizedSearchCV, train_test_split


# --- Define features and target ---
features = [
    'ContinentId',
    'CountryId',
    'CityId',
    'AttractionTypeId',
    'Avg_user_rating',
    'User_visit_count',
    'Attraction_avg_rating',
    'Attraction_popularity'
]
target = 'Rating'

# --- Prepare Data ---
df_model = df[features + [target]].dropna()
X = df_model[features]
y = df_model[target]

# --- Train/Test Split ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Define base model ---
model = CatBoostRegressor(verbose=0, random_state=42)

# --- Define hyperparameter search space ---
param_dist = {
    'iterations': [100, 200, 300],
    'learning_rate': np.linspace(0.01, 0.3, 10),
    'depth': [3, 4, 5, 6, 7, 8, 9, 10],
    'l2_leaf_reg': [1, 3, 5, 7, 9],
    'bagging_temperature': [0, 0.5, 1],
    'border_count': [32, 64, 128]
}

# --- Randomized Search ---
random_search = RandomizedSearchCV(
    model,
    param_distributions=param_dist,
    n_iter=30,
    scoring='r2',
    cv=3,
    verbose=1,
    n_jobs=-1,
    random_state=42
)

random_search.fit(X_train, y_train)

# --- Best parameters and evaluation ---
best_model = random_search.best_estimator_
print("Best Parameters:", random_search.best_params_)

# --- Evaluate on test data ---
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Test MSE: {mse:.2f}")
print(f"Test R^2: {r2:.2f}")


Fitting 3 folds for each of 30 candidates, totalling 90 fits
Best Parameters: {'learning_rate': 0.042222222222222223, 'l2_leaf_reg': 1, 'iterations': 300, 'depth': 4, 'border_count': 64, 'bagging_temperature': 0.5}
Test MSE: 0.23
Test R^2: 0.75
